In [1]:
raise ValueError("Custom message")

ValueError: Custom message

In [2]:
try:
    raise ValueError("Custom message", "some other message")
except ValueError as e:
    print(repr(e))

ValueError('Custom message', 'some other message')


In [3]:
def func_1():
    raise ValueError("Bad Value")

def func_2():
    raise IndexError("Bad Index")


def handle_errors(func):
    try:
        func()
    except ValueError as e:
        print("Handling a ValueError")
        print(e)
    except IndexError as e:
        print("Handling an IndexError")
        print(e)
    

In [4]:
handle_errors(func_1)

Handling a ValueError
Bad Value


In [5]:
handle_errors(func_2)

Handling an IndexError
Bad Index


In [6]:
try:
    raise ValueError("error")
except Exception as e:
    print("Handling Exception", e)
except ValueError as e:
    print("Handling ValueError", e)

Handling Exception error


In [7]:
try:
    raise ValueError("error")
except ValueError as e:
    print("Handling ValueError", e)
except Exception as e:
    print("Handling Exception", e)

Handling ValueError error


In [8]:
try:
    raise ValueError("error")
except ValueError as e:
    print("Handling ValueError", e)
except Exception as e:
    print("Handling Exception", e)
finally:
    print("finally block!")

Handling ValueError error
finally block!


In [9]:
try:
    print("No exceptions")
except Exception:
    print("Exception")
finally:
    print("finally block!")

No exceptions
finally block!


In [10]:
try:
    raise KeyError()
except IndexError:
    print("Exception")
finally:
    print("finally block is running!\n")

finally block is running!



KeyError: 

In [11]:
try:
    raise KeyError()
except KeyError as e:
    print("Handling the KeyError!", e)
    raise ValueError("Oh no, another exception!")
finally:
    print("\nfinally block is running no matter what!\n")

Handling the KeyError! 

finally block is running no matter what!



ValueError: Oh no, another exception!

In [12]:
try:
    raise KeyError()
except KeyError as e:
    print("Handling the KeyError!", e)
else:
    # only gonna run, if no exceptions are present (handled or unhandled)
    print("Else block")

Handling the KeyError! 


In [13]:
try:
    print(1 + 1)
except KeyError as e:
    print("Handling the KeyError!", e)
else:
    print("Else block")

2
Else block


In [14]:
try:
    pass
except ValueError as e:
    print(e)
else:
    print("No exception")

No exception


In [15]:
import json

json_data = """{
    "Alex": {"age": 18},
    "Bob": {"age": 21, "city": "London"},
    "John": {"age": "unknown"}
}"""

data = json.loads(json_data)
data

{'Alex': {'age': 18},
 'Bob': {'age': 21, 'city': 'London'},
 'John': {'age': 'unknown'}}

In [16]:
class Person:
    __slots__ = "name", "_age"

    def __init__(self, name):
        self.name = name
        self._age = None

    @property
    def age(self):
        return self._age

    @age.setter
    def age(self, value):
        if isinstance(value, int) and value >= 0:
            self._age = value
        else:
            raise ValueError("Invalid age")

    def __repr__(self):
        return f"<{type(self).__name__} age={self.age} name={self.name}> @ {hex(id(self))}"

In [17]:

import contextlib
import copy

persons = []
for name, person_details in data.items():
    p = Person(name=name)
    with contextlib.suppress(ValueError):
        p.age = person_details.get("age", 0)
        persons.append(p)

persons

[<Person age=18 name=Alex> @ 0x11090a560,
 <Person age=21 name=Bob> @ 0x11079add0]

In [18]:
persons = []
for name, attributes in data.items():
    try:
        p = Person(name)
        for attrib_name, attrib_value in attributes.items():
            try:
                setattr(p, attrib_name, attrib_value)
            except AttributeError as e:
                print(f"AttributeError: {e} - Ignoring: {name}.{attrib_name}={attrib_value}")
    except ValueError as e:
        print(f"Data for Person({name}) contains an invalid attribute value: {e}")
    else:
        persons.append(p)


AttributeError: 'Person' object has no attribute 'city' - Ignoring: Bob.city=London
Data for Person(John) contains an invalid attribute value: Invalid age


In [19]:
persons

[<Person age=18 name=Alex> @ 0x110930130,
 <Person age=21 name=Bob> @ 0x110829390]

In [20]:
persons = []
for name, attributes in data.items():
    p = Person(name)
    for attrib_name, attrib_value in attributes.items():
        save_person = True
        try:
            setattr(p, attrib_name, attrib_value)
        except AttributeError as e:
            print(f"AttributeError: {e} - Ignoring: {name}.{attrib_name}={attrib_value}")
        except ValueError as e:
            print(f"Data for Person({name}) contains an invalid attribute value: {e}")
            save_person = False
            break

    if save_person:
        persons.append(p)

AttributeError: 'Person' object has no attribute 'city' - Ignoring: Bob.city=London
Data for Person(John) contains an invalid attribute value: Invalid age


In [21]:
persons

[<Person age=18 name=Alex> @ 0x1109318d0,
 <Person age=21 name=Bob> @ 0x1109300a0]

In [22]:
def convert_int(val):
    if not isinstance(val, int):
        raise TypeError()
    if value not in {0, 1}:
        raise ValueError("Integer values 0 or 1 only")
    return bool(val)

def convert_str(val):
    if not isinstance(val, str):
        raise TypeError()

    val = val.casefold()
    if val in {"0", "f", "false"}:
        return False

    if val in {"1", "t", "true"}:
        return True

    raise ValueError("Invalid `bool` string value")


class ConversionError(Exception):
    pass


def make_bool(val):
    try:
        try:
            b = convert_int(val)
        except TypeError:
            try:
                b = convert_str(val)
            except TypeError:
                raise ConversionError("The type is non convertable")

    except ValueError as e:
        raise ConversionError(f"The value {val} can not be converted to a bool: {e}")
    else:
        return b


In [23]:
values = [True, 0, "T", "false", 10, "ABC", 1.0]
for value in values:
    try:
        result = make_bool(value)
    except ConversionError as e:
        result = str(e)
    print(value, result)

True True
0 False
T True
false False
10 The value 10 can not be converted to a bool: Integer values 0 or 1 only
ABC The value ABC can not be converted to a bool: Invalid `bool` string value
1.0 The type is non convertable


In [24]:
def make_bool(val):
    if isinstance(val, int):
        if val in {0, 1}:
            return bool(val)
        else:
            raise ConversionError("Invalid integer value")

    if isinstance(val, str):
        if val.casefold() in {"1", "true", "t"}:
            return True
        if val.casefold() in {"0", "false", "f"}:
            return False
        raise ConversionError("Invalid string value")
    raise ConversionError("Invalid type")
    

In [25]:
values = [True, 0, "T", "false", 10, "ABC", 1.0]
for value in values:
    try:
        result = make_bool(value)
    except ConversionError as e:
        result = str(e)
    print(value, result)

True True
0 False
T True
false False
10 Invalid integer value
ABC Invalid string value
1.0 Invalid type


In [26]:
def get_item_ask_for_forgivenes(seq, idx, default=None):
    try:
        return seq[idx]
    except (IndexError, TypeError, KeyError):
        return default

In [27]:
def get_item_ask_perm(seq, idx, default=None):
    if hasattr(seq, "__getitem__"):
        if idx < len(seq):
            return seq[idx]
    return default

In [28]:
get_item_ask_for_forgivenes([1, 2, 3], 0)

1

In [29]:
get_item_ask_for_forgivenes([1, 2, 3], "abc", default="Not found")

'Not found'

In [30]:
get_item_ask_perm([1, 2, 3], 0)

1

In [31]:
get_item_ask_perm([1, 2, 3], "abc", default="Not found")

TypeError: '<' not supported between instances of 'str' and 'int'